In [1]:
import numpy as np
from matplotlib import pyplot as plt
import json
from tqdm import tqdm
import sys
import corner

sys.path.append("../")

import tdpy

%load_ext autoreload
%autoreload 2

In [2]:
def gen_arr(config, bin=True, n=12000):
    n_secondary = config["n_secondary"]
    l = bin*len(config["binary"].keys()) + n_secondary*len(config["secondary_0"].keys()) + 1
    arr = np.zeros((n, l))*np.nan
    keys = []
    keys.append("m_star")
    for i in range(0, n_secondary):
        for key in config["secondary_0"].keys():
            keys.append(f"{i}/{key}")
    if bin:
        for key in config["binary"].keys():
            keys.append(f"bin/{key}")
    
    return arr, keys




def add_to_array(arr, config, keys, i, bin=True, n=12000):
    if not np.any(arr):
        arr, keys = gen_arr(config, bin=bin, n=n)
        
    n_secondary = config["n_secondary"]
    arr[i, 0] = config["m_star"]
    j = 1
    
    for k in range(0, n_secondary):
        for key in config["secondary_0"].keys():
            arr[i, j] = config[f"secondary_{k}"][key]
            j += 1
    if bin:
        for key in config["binary"].keys():
            arr[i, j] = config["binary"][key]
            j += 1
    
    return arr, keys
    

In [54]:
dir = "sin_inj"
system = "Kepler-62"

keys = None
arr = None
j = 0
stable = np.zeros(12000)

for i in tqdm(range(1, 12000)):
    try:
        with open(f"../output/{dir}/{system}/{i}/config.json") as f:
            cfg = json.load(f)
            arr, keys = add_to_array(arr, cfg, keys, j, bin=False)
            
        elements = np.load(f"../output/{dir}/{system}/{i}/elements.npy")
        a_i = elements[0, :, 0]
        a_f = elements[-1, :, 0]
        stable[i] = np.all(((a_i-a_f)/a_i) < 0.1)
        j += 1

    except Exception as e:
        # print(e)
        pass
arr = arr[:j, :]
stable = stable[:j]
stable = np.logical_not(np.logical_not(stable))

100%|██████████| 11999/11999 [00:06<00:00, 1739.62it/s] 


In [56]:
idxs = np.array(["inc" not in key for key in keys])

param_labels = np.char.replace(np.array(keys)[idxs], "_", "-")



# labels of the populations
listlablpopl = ['not stable', 'stable']

# number of populations
numbpopl = len(listlablpopl)


# number of features
numbfeat = np.sum(idxs)

# generate samples


# labels of the parameters
listlablpara = []
for k in range(numbfeat):
    listlablpara.append([param_labels[k], ''])

typepgrp = 'both'
    
tdpy.plot_grid( [arr[stable, :][: ,idxs], arr[np.logical_not(stable),:][:,idxs]], listlablpara, listlablpopl=listlablpopl, typeplottdim='scat', pathbase="../figs/corner")
# plt.savefig("../figs/corner.pdf")


Number of samples in the populations:
stable
341
not stable
1659
Writing to ../figs/cornerpmar_scat_None.png...


In [40]:
arr.shape

(2000, 37)

In [52]:
arr[stable, :][:, idxs].shape

(14, 31)

In [47]:
.shape

(2000,)